In [1]:
import numpy as np 
import pandas as pd 
import os
import nltk
import re

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip glove.twitter.27B.zip

--2018-11-01 07:41:17--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2018-11-01 07:41:18--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [application/zip]
Saving to: ‘glove.twitter.27B.zip’

glove.twitter.27B.z 100%[===================>]   1.42G  2.91MB/s    in 6m 21s  

2018-11-01 07:47:40 (3.80 MB/s) - ‘glove.twitter.27B.zip’ saved [1520408563/1520408563]

Archive:  glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


In [3]:
from google.colab import files
files.upload()

Saving train.csv to train (1).csv


{'train.csv': b'Scene,Sentiment\n"{Monica} How about, you\xc2\x92re moving!! {Rachel} Look! This is ridiculous. We should be packing you!! {Phoebe} Hey, how are you guys doing? {Rachel} Great! Monica\xc2\x92s moving! {Monica} I am not! {Rachel} Oh really?! Then how come all your stuff is in this box?! {Phoebe} Okay, you guys. You guys I think I know what\xc2\x92s going on here. Okay, you guys {Monica} No Phoebe I am mad! {Phoebe} Well, deep-deep-deep down! {Rachel} Yeah, I\xc2\x92m just mad! {Phoebe} Then keep running. ",NEGATIVE\n{Phoebe} Or we could just follow your clever jokes \xc2\x96 any ideas? ,NEUTRAL\n"{Joey} Hey Rach! Hey, you mind if I read my comic books in here? {Rachel} Sure! Why? {Joey} Oh well, Chandler and Monica are over there and it\'s kinda hard to concentrate. {Rachel} What?! {Rachel} She just called and said that she was gonna be working late! {Rachel} She keeps lying to me! {Rachel} That\'s it! {Rachel} I\'m just gonna go over there and confront them right now! "

In [4]:
!ls

 glove.twitter.27B.100d.txt   sample_data
 glove.twitter.27B.200d.txt  'train (1).csv'
 glove.twitter.27B.25d.txt    train.csv
 glove.twitter.27B.50d.txt    weights4.best.eda.hdf5
 glove.twitter.27B.zip


In [5]:
data = pd.read_csv('train.csv')
pd.set_option('display.max_colwidth',-1)
data.head()

,Scene,Sentiment
0,"{Monica} How about, youre moving!! {Rachel} Look! This is ridiculous. We should be packing you!! {Phoebe} Hey, how are you guys doing? {Rachel} Great! Monicas moving! {Monica} I am not! {Rachel} Oh really?! Then how come all your stuff is in this box?! {Phoebe} Okay, you guys. You guys I think I know whats going on here. Okay, you guys {Monica} No Phoebe I am mad! {Phoebe} Well, deep-deep-deep down! {Rachel} Yeah, Im just mad! {Phoebe} Then keep running.",NEGATIVE
1,{Phoebe} Or we could just follow your clever jokes  any ideas?,NEUTRAL
2,"{Joey} Hey Rach! Hey, you mind if I read my comic books in here? {Rachel} Sure! Why? {Joey} Oh well, Chandler and Monica are over there and it's kinda hard to concentrate. {Rachel} What?! {Rachel} She just called and said that she was gonna be working late! {Rachel} She keeps lying to me! {Rachel} That's it! {Rachel} I'm just gonna go over there and confront them right now!",NEGATIVE
3,"{Joey} You forget how many great songs Heart had. {Chandler} Yeah. {Ross} You know, Barracuda was the first song I learned to play on the keyboard. {Chandler} So, you heard it, you repeated it, so that must mean you wrote it. {Joey} Oh, you guys, with this joke. I gotta say, I know I cracked up, but Im not even sure I got it. {Ross} What, you didnt get it? The doctor is a monkey. {Chandler} And monkeys cant write out prescriptions. {Chandler} You are not allowed to laugh at my joke. {Ross} Your joke? Well, I think the Hef would disagree, which is why he sent me a check for one hundred ah-dollars.",NEUTRAL
4,"{Chandler} Hey Joe! You wanna shoot some hoops? {Joey} Oh no, I cant go. Im practicing; I got an audition to be the host of a new game show.",POSTIVE


In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
scene = data['Scene']

In [0]:
scene2 = []
from nltk.corpus import stopwords
for i in scene:
  replaced = re.sub(r"{.*?}", '', i)
  scene2.append(replaced)

scene3 = []
for j in scene2:
  query = j
  stopwords_list = stopwords.words("english")
  resultwords  = [word for word in re.split("\W+",query) if word.lower() not in stopwords_list]
  result = ' '.join(resultwords)
  scene3.append(result)

In [9]:
num_words = 2000
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
tokenizer.fit_on_texts(scene3)
X = tokenizer.texts_to_sequences(scene3)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 200
X = pad_sequences(X, maxlen=max_length_of_text)

Found 4943 unique tokens.


In [10]:
X[753]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [11]:
conv = {"POSTIVE":0,
       "NEGATIVE":1,
       "MIXED":2,
       "NEUTRAL":3}

for index, row in data.iterrows():
    row['Sentiment'] = conv[row['Sentiment']]
data.head(5)

,Scene,Sentiment
0,"{Monica} How about, youre moving!! {Rachel} Look! This is ridiculous. We should be packing you!! {Phoebe} Hey, how are you guys doing? {Rachel} Great! Monicas moving! {Monica} I am not! {Rachel} Oh really?! Then how come all your stuff is in this box?! {Phoebe} Okay, you guys. You guys I think I know whats going on here. Okay, you guys {Monica} No Phoebe I am mad! {Phoebe} Well, deep-deep-deep down! {Rachel} Yeah, Im just mad! {Phoebe} Then keep running.",1
1,{Phoebe} Or we could just follow your clever jokes  any ideas?,3
2,"{Joey} Hey Rach! Hey, you mind if I read my comic books in here? {Rachel} Sure! Why? {Joey} Oh well, Chandler and Monica are over there and it's kinda hard to concentrate. {Rachel} What?! {Rachel} She just called and said that she was gonna be working late! {Rachel} She keeps lying to me! {Rachel} That's it! {Rachel} I'm just gonna go over there and confront them right now!",1
3,"{Joey} You forget how many great songs Heart had. {Chandler} Yeah. {Ross} You know, Barracuda was the first song I learned to play on the keyboard. {Chandler} So, you heard it, you repeated it, so that must mean you wrote it. {Joey} Oh, you guys, with this joke. I gotta say, I know I cracked up, but Im not even sure I got it. {Ross} What, you didnt get it? The doctor is a monkey. {Chandler} And monkeys cant write out prescriptions. {Chandler} You are not allowed to laugh at my joke. {Ross} Your joke? Well, I think the Hef would disagree, which is why he sent me a check for one hundred ah-dollars.",3
4,"{Chandler} Hey Joe! You wanna shoot some hoops? {Joey} Oh no, I cant go. Im practicing; I got an audition to be the host of a new game show.",0


In [12]:
y = to_categorical(data['Sentiment'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 12)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(800, 200) (800, 4)
(200, 200) (200, 4)


In [13]:
embeddings_index = {}
f = open('glove.twitter.27B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [0]:
embed_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embed_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            embed_dim,
                            weights=[embedding_matrix],
                            input_length=max_length_of_text,
                            trainable=False)

In [0]:
lstm_out = 64
batch_size = 8

In [17]:
inputs = Input((max_length_of_text, ))
x = embedding_layer(inputs)
x = LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(x)
x = Dense(4,activation='sigmoid')(x)
model = Model(inputs, x)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 100)          494400    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 536,900
Trainable params: 42,500
Non-trainable params: 494,400
_________________________________________________________________
None


In [0]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['categorical_accuracy'])

In [0]:
early_stops = EarlyStopping(patience=3, monitor='val_acc')
checkpointer = ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True)

In [27]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = batch_size, epochs = 10, callbacks=[checkpointer], verbose=1)

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 16s 19ms/step - loss: 1.2433 - categorical_accuracy: 0.3837 - val_loss: 1.2104 - val_categorical_accuracy: 0.3600

Epoch 00001: val_loss improved from inf to 1.21044, saving model to weights.hdf5
Epoch 2/10
800/800 [==============================] - 14s 18ms/step - loss: 1.2201 - categorical_accuracy: 0.4000 - val_loss: 1.2219 - val_categorical_accuracy: 0.3600

Epoch 00002: val_loss did not improve from 1.21044
Epoch 3/10
800/800 [==============================] - 16s 20ms/step - loss: 1.2066 - categorical_accuracy: 0.4225 - val_loss: 1.2002 - val_categorical_accuracy: 0.3550

Epoch 00003: val_loss improved from 1.21044 to 1.20023, saving model to weights.hdf5
Epoch 4/10
800/800 [==============================] - 15s 18ms/step - loss: 1.1683 - categorical_accuracy: 0.4412 - val_loss: 1.2346 - val_categorical_accuracy: 0.3950

Epoch 00004: val_loss did not improve from 1.20023
Epoch 5/10

In [0]:
files.download("weights.hdf5")